<span style="color:blue; font-size:40px; line-height:40px; font-family: Calibri;">Udacity Data Analysis Nanodegree</span>

---
<span style="color:blue; font-size:30px; line-height:40px; font-family: Calibri;">Project 3: Wrangle and Analyze Data - Data wrangling report</span>
#### Created by: Juanita Smith
#### Last date: 9 February 2023
---

<img src="../images/weratedogs.png" alt="drawing" width="500"/>

<span style="color:blue; font-size:26px; line-height:40px; font-family: Calibri;">Project Overview</span>

The goal of the project was to demonstrate the data wrangling and analysis skills obtained from the course.

The dataset to be wrangled is the tweet archive of Twitter user [@dog_rates](https://twitter.com/dog_rates), also known as [WeRateDogs](https://en.wikipedia.org/wiki/WeRateDogs). WeRateDogs is a Twitter account that rates people's dogs with a humorous comment about the dog. Dog owners upload a picture of their dog, writes a funny comment and give their own dog a rating out of 10. Owners almost always give their dogs a rating greater than 10. e.g 11/10, 12/10, 13/10, it's the nature of the website. WeRateDogs has over 4 million followers and has received international media coverage. It's only a bit of fun, and brings great joy when browsing the cutest dog images !


### Data wrangling consist of 3 main steps:
>#### **Gather -> Assess -> Clean**

<span style="color:blue; font-size:26px; line-height:40px; font-family: Calibri;">Wrangling Step 1: Data gathering</span>

Real-world data rarely comes clean, especially when it comes from the internet where users enter data into free text fields that are optional, as it's the case with this twitter account.

During the data gathering step, twitter data needed to be downloaded **automaticaly** from the internet using a variety of sources and techniques, with the help of Python and its libraries.

3 data gathering strategies was demonstrated:

### 1. Twitter archive - Automated downloaded from an URL

WeRateDogs downloaded their Twitter archive and sent it to Udacity via email exclusively to use in this project. This archive contains basic tweet data (tweet ID, timestamp, text, etc.) for a sample of 2356 tweets with ratings, for a period between Nov 2015 - July 2017.
Data was automatically downloaded from the Udacity website using `pd.read_csv` and passing the URL as input.

### 2. Use twitter API 'tweepy v2' to retrieve additional data

For each tweet in the twitter archive from step 1, the retweet and like count was retrieved as minimum requirement for the project

The latest version of the tweepy API was used to retrieve the data, which offered more flexibility and data availability, although it took some time to figure out how to use it. I initially tried to download data like geo data, country, place, additional metrics etc, as these could make interesting analytics, however this data was never filled.

```python    
            tweet_fields = ["public_metrics", "entities", "text", "attachments","geo"]
            expansions = ['geo.place_id', 'attachments.media_keys']
            media_fields = ['url', 'type']
            place_fields = ['country_code', 'full_name', 'place_type']
            response = client.get_tweets(tweet_id, 
                                         tweet_fields=tweet_fields, 
                                         expansions=expansions,
                                         media_fields=media_fields,
                                         place_fields = place_fields)
```

Each tweet's entire JSON response was dumped to a txt file, each tweet on its own line

The .txt file was read line by line, and the most useful data to be reused was added to a pandas dataframe. Some data kept are already in twitter archive from step 1, however might be useful to compliment missing or corrupt data potentially. Fields that were kept:

- id (same as tweet_id and were use for table joins)
- text (same text as in twitter archive)         
- retweet_count   
- reply_count     
- like_count      
- quote_count     
- media_url (first image of each tweet was again retrieved to compliment otherwise potential missing images in prediction file)    
- media_type (photo, video, etc)     
- short_url (same url as present in archive data text from step 1, however the url here is stored on it's own) 
- expanded_url (same as in twitter archive)   
- annotation (additional data about dog, contains mostly a place, dog name or dog breed, and can be used to complement missing data)     

Some API errors occurred which was dumped to txt log file. Some tweets are no longer availabe in the API.


### 3. Use the Requests library to download the tweet breed predictions (image_predictions.tsv)

Udacity provided a file that used the dog images to predict the breed of the dog.
Each tweet consist of the link to the image that gave the best predictions, 3 predictions, each with it's confidence level and if the prediction is actually a valid dog breed.

This file (image_predictions.tsv), is hosted on Udacity's servers and was downloaded programmatically using the Requests library and the following URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv


### 4. Download dog profile images (optional extra step)

Image urls containing `.jpg` retrieved from API in step 2 was used to download the first dog image for each tweet from the internet, as additional practice. I had in mind for visualization to show a dog profile card of the most liked breeds and dogs.

<span style="color:blue; font-size:26px; line-height:40px; font-family: Calibri;">Wrangling Step 2: Assessing</span>

**Note:** 
Data obtained are indeed very untidy. The requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues to demonstrate data wrangling skills.
Even though I went beyond this requirement, still not all issues are assessed and cleaned to perfection.
The data were assessed and cleaned, upon which the analysis and visualizations will be based.

Assessing was done using first visual inspection, followed by programmatic inspection for each of the 3 datasets acquired in step 1.
Visual inspection was done by manually inspecting the data after displaying a sample of 10-20 records, using `.sample()`
Programmatic inspection was done using .`info(), nunqiue(), unique(), .duplicated()`. Deeper inspection was done where problems were spotted, using filtering and query techniques.

During the assessment stage, the 'what' is documented, describing only the **observed** problem. The problems were organized by quality vs tidiness issues.

Quality issues were sub organized by completeness, validity, accuracy and consistency, and sorted in order of importance, the same order that will be followed during the cleaning process.

## Summary of assessment

### Quality issues
Do we have all of the records that we should? Do we have missing records or not? Are there specific rows, columns, or cells missing?

<blockquote>
    
#### Completeness Issues (e.g. missing data)

1. twitter_archive: Replies to original tweets exist causing duplicate tweets

2. twitter_archive: Retweets exist causing duplicate tweets

3. twitter_archive: `name` contains string value 'None' indicating missing data

4. twitter_archive: Columns `doggo, floofer, pupper, puppo` contain value 'None' indicating missing data

5. twitter_archive: some `expanded_urls` are duplicated (it might be because of retweets and replies)


#### Validity Issues (incorrect entries, data types)

6. twitter_archive: 23 records (prior start of cleaning) have ratings_denominator values other than 10
7. twitter_archive: `expanded_urls` contains more than one url sometimes, example tweet id 853299958564483072


#### Accuracy Issues

8. twitter_archive: `name` starting with lower case are not valid dog names e.g. 'the, 'actually', 'this', 'one'


#### Consistency

9. twitter_api: 'id' column should be called 'tweet_id' to be consistent with twitter_archive table.
  
</blockquote>

### Tidiness issues

<blockquote>
    
#### Twitter archive table:

1. `timestamp` should be type datetime

2. `source` contain multiple pieces of information: url and source

3. `text` contains multiple information: text, score and url

4.  Columns exist with 90% missing data: `in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp`

5. `doggo, floofer, pupper, puppo` should be converted into one column to reflect the dog stage

#### `twitter_api` table:

6. `retweet_count`, `like_count`, `media_url` and `annotation` columns should be merged with selected fields from twitter archive table to build a new master table avoiding duplicated columns

#### Twitter predictions table:

7.  The best dog breed should be selected and be part the new twitter master table
8.  Breed name are badly formatted, it contains '_' instead of spaces and each word can be capitalized.

</blockquote>

<span style="color:blue; font-size:26px; line-height:40px; font-family: Calibri;">Wrangling Step 3: Cleaning</span>

First, copies were made of each dataset, which was further used to perform the cleaning actions on.

Cleaning was done following the framework **Define -> Code -> Test** for each issue found during assessing one-by-one

1. **Define**: write pseudocode **how** the issue will be fixed, normally the define statement start with a verb
2. **Code**: implement the plan defined
3. Immediately **test** the code after implementation to ensure the problem no longer exist. In most cases the `assert` statement was used for an automated testing approach. In some cases `.head()` or `.sample()` was used to manually inspect if the problem was solved

Example:

<img src="../images/cleanexample.png" alt="cleanexample" width="900" style="float:left"/>

At the end of the cleaning step, we have a new master twitter archive dataset exported as `.csv` file, containing no duplicate columns, with the best data merged from all 3 datasets.
Only original tweets with images were kept.

#### final result:  twitter_archive_master.csv

| Column  | From table | Additional comments |  
| :- | :- |  :- |
| tweet_id | twitter_archive |  Only records were kept which could be inner joined with `twitter_prediction` table. It was a project requirement that tweet should contain an image which was present in the twitter_prediction table
| timestamp | twitter_archive | Contain only date YYYYMMDD, time was dropped
| source | twitter_archive |  Only source text was extracted (e.g. Twitter for iPhone), URL part and other HTML characters was dropped
| expanded_urls | twitter_archive | Full URL link to web page where photo is located. Note: this link does not contain link to actual image, it does not end with `.jpg`.
| rating_numerator | twitter_archive |  Normally above 10. Some outliers were detected above 250, however it was decided to leave those records, as it's the owner themselves giving this rating as thus a bias rating anyhow.
| rating_denominator | twitter_archive | Now only contains value of 10, other records were dropped as errors were mostly due to extraction issues where text contained more than 1 score pattern e.g. `xx/xx`
| name | twitter_archive | A lot of dog names are missing, 'None' was converted to `None`
| cleantext | twitter_archive | Text part was extracted, with score and url dropped. Note: When score is in middle of the text, texts after score are dropped in exceptation cases. 
| dog_stage | twitter_archive |  Only 368 out of 2071 tweets have dog stages, after all the cleaning data quality is still not great. All dog stage columns were merged into 1 column and further enhanced to re-extract dog stages from text to enhance cases where dog stage was missing.
| retweet_count | twitter_api | Amount of times an original tweet was retweeted, shows popularity 
| like_count | twitter_api |  Amount of times an original tweet was liked, shows popularity 
| media_url | twitter_api |  URL link to first profile picture containing `.jpg` which can be used to download imagines
| short_url | twitter_api |  Short URL link to tweet web page, it's the same url that was present in `twitter_archive-text` but now it's stored seperately
| annotation | twitter_api |  First annotion, contains in most cases a dog name, breed or place
| breed | twitter_prediction |  Breed with highest prediction score above 5% was selected, where the breed is an actual dog breed
| jpg_url | twitter_prediction | URL link to image ending with `.jpg` giving the highest breed prediction store

In [1]:
# convert notebook to html
import os
os.system('jupyter nbconvert --to html wrangle_report.ipynb')

[NbConvertApp] WARNING | Config option `kernel_spec_manager_class` not recognized by `NbConvertApp`.
[NbConvertApp] Converting notebook wrangle_report.ipynb to html
[NbConvertApp] Writing 600750 bytes to wrangle_report.html


0